# Optimizing Schwefel's Function

## This code finds optima of Schwefel's function. Schwefel's function has many optima and we want to find a lot of them. 

In [ ]:
import numpy as np
from hgdl.hgdl import HGDL as hgdl
from hgdl.support_functions import *
import time
import dask.distributed as distributed
import tracemalloc




arr  = 5
brr  = 6
bounds = np.array([[-500,500],[-500,500]])
#dask_client = distributed.Client("10.0.0.184:8786")
a = hgdl(schwefel, schwefel_gradient, bounds,
            hess = schwefel_hessian,
            #global_optimizer = "random",
            global_optimizer = "genetic",
            #global_optimizer = "gauss",
            local_optimizer = "dNewton",
            number_of_optima = 30000,
            args = (arr,brr), radius = None, num_epochs = 100)

x0 = np.random.uniform(low = bounds[:, 0], high = bounds[:,1],size = (20,2))
print("starting positions: ")
print(x0)
print("--------------------")
a.optimize(x0 = x0)
    

print("main thread submitted HGDL and will now sleep for 2 seconds")
time.sleep(2)
print("main thread asks for 10 best solutions:")
print(a.get_latest())
#a.cancel_tasks()
print("main sleeps for another 2 seconds")
time.sleep(2)
print("main thread kills optimization")
res = a.kill_client()
print("hgdl was killed but I am waiting 2s")
print("")
print("")
print("")
print("")
print(res)


## How did we do? 
For the plot to work, you will need plotly. No big deal, activate your environment and type "pip install plotly", then restart the notebook.

In [ ]:
import numpy as np
import plotly.graph_objects as go

def plot(x,y,z,data = None):
    fig = go.Figure()
    fig.add_trace(go.Surface(x = x, y = y,z=z))
    if data is not None: 
        fig.add_trace(go.Scatter3d(x=data[:,0], y=data[:,1], z=data[:,2],
                                   mode='markers')) 

    fig.update_layout(title='Surface Plot', autosize=True,
                  width=800, height=800, font=dict(
                  family="Courier New, monospace",
                  size=18),
                  margin=dict(l=65, r=50, b=65, t=90))
    fig.show()

In [ ]:
x1,x2 = np.linspace(-500,500,100),np.linspace(-500,500,100)
x_pred = np.transpose([np.tile(x1, len(x2)), np.repeat(x2, len(x1))])
z = np.zeros((10000))
for i in range(10000): 
    z[i] = schwefel(x_pred[i], 1, 1)
data = np.column_stack([res["x"],res["f(x)"].reshape(-1,1)])
plot(x1,x2,z.reshape(100,100).T, data = data)    